In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import Counter

In [2]:
data = pd.read_csv('data/Project1_data.csv')

f = open('data/out.pkl', 'r')
train = pickle.load(f) 
f.close()

g = open('data/ages.pkl', 'r')
ages = pickle.load(g) 
g.close()

In [3]:
ids   = list(set(data['Id'].values.tolist()))
ids_t = list(set(train['Id'].values.tolist()))
ids_d = list(set(ids)^set(ids_t))

d = {'Id' : ids}
Y1_2017 = pd.DataFrame(d, columns=['Id', 'Age', 'Sex', 'AvgTime','NumRaces','YrsSinceLast','Y1'])
#Y1_2017['Age']=0.
Y1_2017['NumRaces']=0.
Y1_2017['YrsSinceLast']=0.

In [4]:
for i in Y1_2017.index.tolist():
    rid = Y1_2017.loc[i]['Id']
    
    if rid in ids_t:
        age = ages[str(rid)] - ages[str(rid)]%5
        sex = train[train['Id']==rid]['Sex'].values[0]

        prevRaces = train[(train['Id']==rid)&(train['Year']<2017)]
        prevTimeSum = sum(prevRaces['Time'].values)
        numRaces  = prevRaces['Year'].size
        avgTime = ((prevTimeSum-7942.)/(30282.-7942.))/numRaces

        if numRaces == 0:
            ysl = 15
        else:
            prev_year = max(train[(train['Id']==rid)&(train['Year']<2017)]['Year'].values.tolist())
            ysl = 2017-prev_year

        Y1_2017.set_value(i,'Age',age)
        Y1_2017.set_value(i,'Sex',sex)
        Y1_2017.set_value(i,'AvgTime', avgTime)
        Y1_2017.set_value(i,'NumRaces', numRaces)
        Y1_2017.set_value(i,'YrsSinceLast',ysl)

In [5]:
Y1_2017 = Y1_2017.fillna(-1.)

## Runners Who Ran in 2013

In [6]:
rid_2013_and_other_years = []
rid_2013_only = []
rid_others = []
for i in Y1_2017[Y1_2017['Age']==(-1)].index.tolist():
    rid = Y1_2017.loc[i]['Id']
    if(data[(data['Id']==rid)&(data['Year']==2013)]['Id'].size > 0):
        if( len(set(data[data['Id']==rid]['Year'].values.tolist())) > 1 ):
            rid_2013_and_other_years.append(rid)
        else:
            rid_2013_only.append(rid)
    else:
        rid_others.append(rid)
        
data_2013 = pd.read_csv('data/data.csv')

In [7]:
def fullMarathonTime(halfMarathonTime):
    return np.round(halfMarathonTime*((26.219/13.095)**1.06))

In [8]:
time_2013 = data_2013[data['Year']==2013]['Time'].values.tolist()
time_2013_max = 30282.#fullMarathonTime( max(time_2013) )
time_2013_min = 7942.#fullMarathonTime( min(time_2013) )

In [9]:
Y1_2017[Y1_2017['Age']==(-1)]['Id'].size

2980

In [10]:
for rid in rid_2013_only:
    i = Y1_2017[Y1_2017['Id']==rid].index.tolist()[0]

    if (len(data[data['Id']==rid]['Year'].values.tolist())==1):
        record = data_2013[data_2013['Id']==rid]
        age = 2017- record['Year']+record['Age']
        age = age - age%5
        sex = record['Sex']
        time = (fullMarathonTime ( record['Time'] ) - time_2013_min)/ (time_2013_max- time_2013_min)
        numRaces = 1.
        ysl = 2017-record['Year']
        
        Y1_2017.set_value(i,'Age',age)
        Y1_2017.set_value(i,'Sex',sex)
        Y1_2017.set_value(i,'AvgTime', time)
        Y1_2017.set_value(i,'NumRaces', numRaces)
        Y1_2017.set_value(i,'YrsSinceLast',ysl)
    else:
        record = data_2013[data_2013['Id']==rid]
        size = record['Id'].size
        
        avg_age = np.round(sum(record['Age'].values.tolist())*1./size)
        age = 2017- 2013 + avg_age
        age = age - age%5
        
        sex = sum(record['Sex'].values.tolist())*1./size
        if(sex > 0.5):
            sex = 1
        else:
            sex = 0
        
        avg = np.round(sum(record['Time'].values.tolist())*1./size)
        time = (fullMarathonTime ( avg ) - time_2013_min)/ (time_2013_max- time_2013_min)
        numRaces = 1.
        ysl = 2017-max(list(set(data_2013[data_2013['Id']==rid]['Year'].values.tolist())))
        
        Y1_2017.set_value(i,'Age',age)
        Y1_2017.set_value(i,'Sex',sex)
        Y1_2017.set_value(i,'AvgTime', time)
        Y1_2017.set_value(i,'NumRaces', numRaces)
        Y1_2017.set_value(i,'YrsSinceLast',ysl)

In [11]:
Y1_2017[Y1_2017['Age']==(-1)]['Id'].size

33

In [12]:
for rid in rid_2013_and_other_years:
    i = Y1_2017[Y1_2017['Id']==rid].index.tolist()[0]
    record = data_2013[data_2013['Id']==rid]
    size = record['Id'].size
        
    avg_age = np.round(sum(record['Age'].values.tolist())*1./size)
    avg_year= np.round(sum(record['Year'].values.tolist())*1./size)
    
    age = 2017- avg_year + avg_age
    age = age - age%5

    sex = sum(record['Sex'].values.tolist())*1./size
    if(sex > 0.5):
        sex = 1
    else:
        sex = 0

    avg_time = np.round(sum(record['Time'].values.tolist())*1./size)
    time = (fullMarathonTime ( avg ) - time_2013_min)/ (time_2013_max- time_2013_min)
    numRaces = 1.25841017604
    #numRaces = len(set(data_2013[data_2013['Id']==rid]['Year'].values.tolist()))*1.
    ysl = 2017-max(list(set(data_2013[data_2013['Id']==rid]['Year'].values.tolist())))
    
    Y1_2017.set_value(i,'Age',age)
    Y1_2017.set_value(i,'Sex',sex)
    Y1_2017.set_value(i,'AvgTime', time)
    Y1_2017.set_value(i,'NumRaces', numRaces)
    Y1_2017.set_value(i,'YrsSinceLast',ysl)

In [13]:
Y1_2017[Y1_2017['Age']==(-1)]['Id'].size

30

In [14]:
for rid in rid_others:
    i = Y1_2017[Y1_2017['Id']==rid].index.tolist()[0]
    record = data_2013[data_2013['Id']==rid]
    
    if (data[data['Id']==rid]['Id'].size==1):
        record = data_2013[data_2013['Id']==rid]
        age = 2017- record['Year']+record['Age']
        age = age - age%5
        sex = record['Sex']
        time = (record['Time'] - time_2013_min)/ (time_2013_max- time_2013_min)
        numRaces = 1.
        ysl = 2017-record['Year']
        
        Y1_2017.set_value(i,'Age',age)
        Y1_2017.set_value(i,'Sex',sex)
        Y1_2017.set_value(i,'AvgTime', time)
        Y1_2017.set_value(i,'NumRaces', numRaces)
        Y1_2017.set_value(i,'YrsSinceLast',ysl)
    else:
        size = record['Id'].size
        avg_age = np.round(sum(record['Age'].values.tolist())*1./size)
        avg_year= np.round(sum(record['Year'].values.tolist())*1./size)

        age = 2017- avg_year + avg_age
        age = age - age%5

        sex = sum(record['Sex'].values.tolist())*1./size
        if(sex > 0.5):
            sex = 1
        else:
            sex = 0

        avg_time = np.round(sum(record['Time'].values.tolist())*1./size)
        time = (avg_time - time_2013_min)/ (time_2013_max- time_2013_min)
        
        numRaces = len(set(data_2013[data_2013['Id']==rid]['Year'].values.tolist()))*1.
        ysl = 2017-max(list(set(data_2013[data_2013['Id']==rid]['Year'].values.tolist())))
        
        Y1_2017.set_value(i,'Age',age)
        Y1_2017.set_value(i,'Sex',sex)
        Y1_2017.set_value(i,'AvgTime', time)
        Y1_2017.set_value(i,'NumRaces', numRaces)
        Y1_2017.set_value(i,'YrsSinceLast',ysl)

In [15]:
Y1_2017[Y1_2017['Age']==(-1)]['Id'].size

0

In [16]:
for i in Y1_2017.index.tolist():
    d   = Y1_2017.loc[i]
    age = d['Age']
    numRaces = d['NumRaces']
    ysl = d['YrsSinceLast']
    
    Y1_2017.set_value(i,'Age',(age-11.)/(92.-11.))
    Y1_2017.set_value(i,'NumRaces', (numRaces-1.)/(14.))
    Y1_2017.set_value(i,'YrsSinceLast',(ysl-1.)/(13.))

In [17]:
Y1_2017

,Id,Age,Sex,AvgTime,NumRaces,YrsSinceLast,Y1
0,1,0.234568,1.0,0.329364,0.000000,0.000000,-1.0
1,2,0.604938,1.0,0.465533,0.000000,0.000000,-1.0
2,3,0.234568,0.0,0.462489,0.000000,0.000000,-1.0
3,4,0.296296,1.0,0.556065,0.071429,0.000000,-1.0
4,5,0.358025,0.0,0.846757,0.571429,0.000000,-1.0
5,6,0.419753,1.0,0.402910,0.000000,0.000000,-1.0
6,7,0.358025,1.0,0.441003,0.071429,0.000000,-1.0
7,8,0.234568,0.0,0.287019,0.000000,0.000000,-1.0
8,9,0.481481,1.0,0.677026,0.214286,0.000000,-1.0
9,10,0.172840,1.0,0.413205,0.000000,0.000000,-1.0


In [60]:
g = open('data/xy1_2017.pkl','w')
pickle.dump(Y1_2017,g)
g.close()

Y1_2017.to_csv('data/xy1_2017.csv')